# EXPERIMENTO FALLIDO 

No es capaz de hacer detección de objetos. Probablemente el modelo preentrenado no sea adecuado

In [5]:
import sys
import cv2
import os
import shutil

sys.path.append(os.path.join(os.getcwd(), ".."))

from Extra.parse_xml import parse_xml
from Extra.translate_coco_to_images_net import translate_coco_to_images_net

import matplotlib.pyplot as plt
import numpy as np

In [1]:
def clear_directory(directory_path):
    if os.path.exists(directory_path):
        shutil.rmtree(directory_path)
    os.makedirs(directory_path)

In [16]:
def find_xml_file(path, jpeg_filename):
    # Get the base filename without extension
    base_filename = os.path.splitext(os.path.basename(jpeg_filename))[0]

    # Construct the XML filename
    xml_filename = os.path.join(path, f"{base_filename}.xml")

    # Check if the XML file exists
    if os.path.exists(xml_filename):
        return xml_filename
    else:
        return None

In [17]:
def calculate_iou(box1, box2):
    # Calculate the intersection coordinates
    inter_xmin = max(box1[0], box2[0])
    inter_ymin = max(box1[1], box2[1])
    inter_xmax = min(box1[2], box2[2])
    inter_ymax = min(box1[3], box2[3])

    # Calculate the area of intersection
    inter_area = max(0, inter_xmax - inter_xmin + 1) * max(0, inter_ymax - inter_ymin + 1)

    # Calculate the area of both boxes
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)

    # Calculate IoU
    iou = inter_area / float(box1_area + box2_area - inter_area)
    return iou

def calculate_absolute_difference(box1, box2):
    # Calculate absolute differences for each coordinate
    diff_xmin = abs(box1[0] - box2[0])
    diff_ymin = abs(box1[1] - box2[1])
    diff_xmax = abs(box1[2] - box2[2])
    diff_ymax = abs(box1[3] - box2[3])

    # You can sum up the absolute differences or use any other measure
    total_absolute_difference = diff_xmin + diff_ymin + diff_xmax + diff_ymax
    return total_absolute_difference

In [18]:
resources_directory = "../../resources/"

# Load COCO class labels
class_labels = []
with open(resources_directory + "coco.names", "rt") as f:
    class_labels = f.read().rstrip('\n').split('\n')

# Load the pre-trained Faster R-CNN model and configuration
model_path = resources_directory + 'frozen_inference_graph.pb'
config_path = resources_directory + 'faster_rcnn_inception_v2_coco_2018_01_28.pbtxt'
net = cv2.dnn.readNetFromTensorflow(model_path, config_path)

In [19]:
folder_path = "../../testImagesNet/Data/"
annotations_directory = "../../testImagesNet/Annotations/"
output_directory = "../../output/"

clear_directory(output_directory)

image_list = []
image_path_list = []

# List all files in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(('.jpg', '.png', '.jpeg', '.JPEG')):  # You can add more extensions if needed
        # Construct the full file path
        file_path = os.path.join(folder_path, filename)
        
        # Read the image and append it to the list
        img = cv2.imread(file_path)
        image_list.append(img)
        image_path_list.append(file_path)

n = 0
results = {}
for image in image_list:

    xml_data = parse_xml(find_xml_file(annotations_directory, image_path_list[n]))

    # Create a 4D blob from the image (resize, normalize, swap channels)
    blob = cv2.dnn.blobFromImage(image, scalefactor=1/255.0, size=(416, 416), swapRB=True, crop=False)

    # Set the input to the network
    net.setInput(blob)

    # Get output layer names
    out_layer_names = net.getUnconnectedOutLayersNames()

    # Forward pass to get output
    output = net.forward(out_layer_names)

    # Perform object detection
    detections = net.forward()

    # Loop through the detections
    for target in xml_data:
        iou = 0

        searched_label = translate_coco_to_images_net(target["name"])

        image_iou_dict = {}
        if searched_label in results:
            image_iou_dict = results[searched_label]
        if image_path_list[n] not in image_iou_dict:
            image_iou_dict[image_path_list[n]] = []

        for detection in detections[0, 0, :, :]:

            score = float(detection[2])
            class_id = int(detection[1])
            
            if score > 0.5 and class_id < len(class_labels):  # Filter detections with confidence greater than 50%
                height, width, _ = image.shape
                xmin = int(detection[3] * width)
                ymin = int(detection[6] * height)
                ymax = int(detection[4] * height)
                xmax = int(detection[5] * width)

                # Draw bounding box and label on the image
                cv2.rectangle(image, (xmin, ymax), (xmax, ymin), (0, 255, 0), 2)
                label = f'{class_labels[class_id]}: {score:.2f}'
                cv2.putText(image, label, (xmin, ymax - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                if (class_id == searched_label):
                    box1 = [xmin, ymin, xmax, ymax]
                    box2 = [target["xmin"], target["ymin"], target["xmax"], target["ymax"]]

                    new_iou = calculate_iou(box1, box2)

                    if new_iou > iou:
                        iou = new_iou
                        
        image_iou_dict[image_path_list[n]].append(iou)
        results[searched_label] = image_iou_dict

    # Save the annotated image
    filename = os.path.join(output_directory, os.path.basename(image_path_list[n]))
    cv2.imwrite(filename, image)

    n += 1

In [20]:
print(results["dog"])
print(results["car"])
print(results["apple"])

{'../../testImagesNet/Data/n02089973_1.JPEG': [0], '../../testImagesNet/Data/n02089973_13.JPEG': [0, 0], '../../testImagesNet/Data/n02089973_2.JPEG': [0], '../../testImagesNet/Data/n02089973_26.JPEG': [0], '../../testImagesNet/Data/n02093859_10.JPEG': [0], '../../testImagesNet/Data/n02093859_3.JPEG': [0], '../../testImagesNet/Data/n02097209_1.JPEG': [0], '../../testImagesNet/Data/n02097209_12.JPEG': [0, 0], '../../testImagesNet/Data/n02097209_8.JPEG': [0], '../../testImagesNet/Data/n02105855_5.JPEG': [0], '../../testImagesNet/Data/n02111277_25.JPEG': [0], '../../testImagesNet/Data/n02112018_2991.JPEG': [0], '../../testImagesNet/Data/n02112018_3024.JPEG': [0], '../../testImagesNet/Data/n02112018_3033.JPEG': [0], '../../testImagesNet/Data/n02112018_3054.JPEG': [0], '../../testImagesNet/Data/n02112018_8.JPEG': [0], '../../testImagesNet/Data/n02114548_12.JPEG': [0], '../../testImagesNet/Data/n02114548_34.JPEG': [0], '../../testImagesNet/Data/n02114548_47.JPEG': [0]}
{'../../testImagesNet/D

In [8]:
# Display the result
cv2.imshow('Object Detection', image)
cv2.waitKey(0)
cv2.destroyAllWindows()